In [33]:

# coding: utf-8

# In[ ]:


from __future__ import division


# In[ ]:


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
import cv2
import os
from skimage import io as ski_io
from skimage.viewer import ImageViewer
from skimage import color as ski_color
from skimage import io as ski_io
import time
from nolearn.lasagne import BatchIterator
from skimage import transform as ski_transform
NUM_CLASS=10
def one_hot(labels, n_class = NUM_CLASS):
    total_record = len(labels)
    y = np.zeros((total_record, n_class))
    y[np.arange(total_record), labels] = 1
    return y


from collections import namedtuple

Parameters = namedtuple('Parameters', [
        # Data parameters
        'num_classes', 'image_size', 
        # Training parameters
        'batch_size', 'max_epochs', 'log_epoch', 'print_epoch',
        # Optimisations
        'learning_rate_decay', 'learning_rate',
        'l2_reg_enabled', 'l2_lambda', 
        'early_stopping_enabled', 'early_stopping_patience', 
        'resume_training', 
        # Layers architecture
        'conv1_k', 'conv1_d', 'conv1_p', 
        'conv2_k', 'conv2_d', 'conv2_p', 
        'conv3_k', 'conv3_d', 'conv3_p', 
        'fc4_size', 'fc4_p'
    ])

import os

class Paths(object):
    """
    Provides easy access to common paths we use for persisting 
    the data associated with model training.
    """
    
    def __init__(self, params):
        """
        Initialises a new `Paths` instance and creates corresponding folders if needed.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
        """
        self.model_name = self.get_model_name(params)
        self.var_scope = self.get_variables_scope(params)
        self.root_path = os.getcwd() + "/models/" + self.model_name + "/"
        self.model_path = self.get_model_path()
        self.train_history_path = self.get_train_history_path()
        self.learning_curves_path = self.get_learning_curves_path()
        if not os.path.exists(self.root_path):
            os.makedirs(self.root_path)

    def get_model_name(self, params):
        """
        Generates a model name with some of the crucial model parameters encoded into the name.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Model name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        print ("call get model name")
        model_name = "k{}d{}p{}_k{}d{}p{}_k{}d{}p{}_fc{}p{}".format(
            params.conv1_k, params.conv1_d, params.conv1_p, 
            params.conv2_k, params.conv2_d, params.conv2_p, 
            params.conv3_k, params.conv3_d, params.conv3_p, 
            params.fc4_size, params.fc4_p
        )
        model_name += "_lrdec" if params.learning_rate_decay else "_no-lrdec"
        model_name += "_l2" if params.l2_reg_enabled else "_no-l2"
        return model_name

    def get_variables_scope(self, params):
        """
        Generates a model variable scope with some of the crucial model parameters encoded.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Variables scope name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        var_scope = "k{}d{}_k{}d{}_k{}d{}_fc{}_fc0".format(
            params.conv1_k, params.conv1_d,
            params.conv2_k, params.conv2_d,
            params.conv3_k, params.conv3_d, 
            params.fc4_size
        )
        return var_scope

    def get_model_path(self):
        """
        Generates path to the model file.
   
        Returns
        -------
        Model file path.
        """
        return self.root_path + "model.ckpt"

    def get_train_history_path(self):
        """
        Generates path to the train history file.
   
        Returns
        -------
        Train history file path.
        """
        return self.root_path + "train_history"
    
    def get_learning_curves_path(self):
        """
        Generates path to the learning curves graph file.
   
        Returns
        -------
        Learning curves file path.
        """
        return self.root_path + "learning_curves.png"


# In[ ]:


def fully_connected(input, size):
    """
    Performs a single fully connected layer pass, e.g. returns `input * weights + bias`.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer = tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    return tf.nn.relu(fully_connected(input, size))

def conv_relu(input, kernel_size, depth):
    """
    Performs a single convolution layer pass.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer = tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides = [1, 1, 1, 1], padding = 'SAME')
    return tf.nn.relu(conv + biases)

def pool(input, size):
    """
    Performs a max pooling layer pass.
    """
    return tf.nn.max_pool(
        input, 
        ksize = [1, size, size, 1], 
        strides = [1, size, size, 1], 
        padding = 'SAME'
    )

def model_pass(input, params, is_training):
    # Convolutions

    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = params.conv1_k, depth = params.conv1_d) 
    with tf.variable_scope('pool1'): 
        pool1 = pool(conv1, size = 2)
        pool1 = tf.cond(is_training, lambda: tf.nn.dropout(pool1, keep_prob = params.conv1_p), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = params.conv2_k, depth = params.conv2_d)
    with tf.variable_scope('pool2'):
        pool2 = pool(conv2, size = 2)
        pool2 = tf.cond(is_training, lambda: tf.nn.dropout(pool2, keep_prob = params.conv2_p), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = params.conv3_k, depth = params.conv3_d)
    with tf.variable_scope('pool3'):
        pool3 = pool(conv3, size = 2)
        pool3 = tf.cond(is_training, lambda: tf.nn.dropout(pool3, keep_prob = params.conv3_p), lambda: pool3)
    
    # Fully connected
    
    # 1st stage output
    pool1 = pool(pool1, size = 4)
    shape = pool1.get_shape().as_list()
    pool1 = tf.reshape(pool1, [-1, shape[1] * shape[2] * shape[3]])
    
    # 2nd stage output
    pool2 = pool(pool2, size = 2)
    shape = pool2.get_shape().as_list()
    pool2 = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])    
    
    # 3rd stage output
    shape = pool3.get_shape().as_list()
    pool3 = tf.reshape(pool3, [-1, shape[1] * shape[2] * shape[3]])
    
    flattened = tf.concat([pool1, pool2, pool3], 1)
    
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = params.fc4_size)
        fc4 = tf.cond(is_training, lambda: tf.nn.dropout(fc4, keep_prob = params.fc4_p), lambda: fc4)
    with tf.variable_scope('out'):
        logits = fully_connected(fc4, size = params.num_classes)
    return logits


parameters = Parameters(
    # Data parameters
    num_classes = NUM_CLASS,
    image_size = (32, 32),
    # Training parameters
    batch_size = 256,
    max_epochs = 100,
    log_epoch = 1,
    print_epoch = 1,
    # Optimisations
    learning_rate_decay = False,
    learning_rate = 0.0001,
    l2_reg_enabled = True,
    l2_lambda = 0.0001,
    early_stopping_enabled = True,
    early_stopping_patience = 100,
    resume_training = True,
    # Layers architecture
    conv1_k = 5, conv1_d = 32, conv1_p = 0.9,
    conv2_k = 5, conv2_d = 64, conv2_p = 0.8,
    conv3_k = 5, conv3_d = 128, conv3_p = 0.7,
    fc4_size = 1024, fc4_p = 0.5
)


def get_top_k_predictions(params, X, k = 5):
    
    # Initialisation routines: generate variable scope, create logger, note start time.
    paths = Paths(params)
    
    # Build the graph
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
        tf_x = tf.placeholder(tf.float32, shape = (None, params.image_size[0], params.image_size[1], 1))
        is_training = tf.constant(False)
        with tf.variable_scope(paths.var_scope):
            predictions = tf.nn.softmax(model_pass(tf_x, params, is_training))
            top_k_predictions = tf.nn.top_k(predictions, k)

    with tf.Session(graph = graph) as session:
        session.run(tf.global_variables_initializer())
        tf.train.Saver().restore(session, paths.model_path)
        [p] = session.run([top_k_predictions], feed_dict = {
                tf_x : X
            }
        )
        return np.array(p)


In [34]:
import cv2
import numpy as np

In [35]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [36]:
map_class = ['stop', 'turn_left', 'turn_right', 'prob_left', 'prob_right', 'no_entry', 'speed_limit', 'min_speed', 'end_min', 'unknown']
map_answer = []

In [38]:
import cv2
import numpy as np
cap = cv2.VideoCapture('./input-sample/MVI_1049.avi')

# Initialisation routines: generate variable scope, create logger, note start time.
paths = Paths(parameters)

# Build the graph
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_x = tf.placeholder(tf.float32, shape = (None, parameters.image_size[0], parameters.image_size[1], 1))
    is_training = tf.constant(False)
    with tf.variable_scope(paths.var_scope):
        predictions = tf.nn.softmax(model_pass(tf_x, parameters, is_training))
        top_k_predictions = tf.nn.top_k(predictions, 5)

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    tf.train.Saver().restore(session, paths.model_path)        
    while (1):

        _, frame = cap.read()
#         frame = cv2.resize(frame, (640, 360), interpolation = cv2.INTER_LINEAR)

        blur = cv2.medianBlur(frame.copy(), 5)

        # Convert BGR to HSV
        hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        # define range of blue color in HSV
        lower_blue = np.array([100, 70, 40])
        upper_blue = np.array([130, 255, 255])

        lower_red = np.array([0, 70, 50])
        upper_red = np.array([10, 255, 255]) 
        r_mask1 = cv2.inRange(hsv, lower_red, upper_red) 
        r_mask2 = cv2.inRange(hsv, np.array([100, 10, 170]), np.array([180, 230, 255])) 
        r_mask3 = cv2.inRange(hsv, np.array([157, 60, 11]), np.array([180, 255, 255])) 
        red_mask = r_mask1 | r_mask2 | r_mask3
        

        # Threshold the HSV image to get only blue colors
        blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
        mask = blue_mask + red_mask

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(frame, frame, mask=mask)
        mask = cv2.erode(mask.copy(), None, iterations=2)
        mask2 = cv2.dilate(mask.copy(), None, iterations=2)

        cnts = cv2.findContours(mask2.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)[-2]
        framedraw = frame.copy()
        if len(cnts) > 0:
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
#                 Vi tri cac bien            
                if w < 20 or h < 20:
                    cv2.rectangle(framedraw, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    continue
                roi = frame[y:y+h, x:x+w]
                # extract image and feed to cnn
                roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)  
                roi = ski_transform.resize(roi, (32, 32))
                roi = ski_color.rgb2gray(roi)
                roi = roi.reshape(roi.shape + (1,))        
                tf_X = np.array([roi])
                cv2.rectangle(framedraw, (x, y), (x + w, y + h), (255,0,0), 2)
                [prediction] = session.run([top_k_predictions], feed_dict = {
                        tf_x : tf_X
                    }
                )
                prob_class =prediction[0][0][0] 
                class_ord = prediction[1][0][0]
                class_name = map_class[class_ord]
                cv2.putText(framedraw, class_name, (x+w, y + int(h / 2)), cv2.FONT_HERSHEY_SIMPLEX, 1,color=(0,0,255))
                print (prob_class)
#         cv2.imshow('frame', framedraw)

#         k = cv2.waitKey(5) & 0xFF
#         if k == 27:
#             break

cv2.destroyAllWindows()

call get model name
INFO:tensorflow:Restoring parameters from /home/tuyenlv17/data/programming/projects/digital-race-2/localization/models/k5d32p0.9_k5d64p0.8_k5d128p0.7_fc1024p0.5_no-lrdec_l2/model.ckpt
1.0
0.97570634
0.989888
0.9181425
0.46181133
0.9746753
0.8130978


/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


1.0
0.94956625
0.84789383
0.96293783
0.72437376
1.0
0.63565165
0.8104815
0.9566182
0.52081007
1.0
0.9632413
0.7865575
0.83145297
0.9624225
0.6867899
1.0
0.50010425
0.97568965
0.99400204
0.8930996
0.94021714
0.7406435
1.0
0.6980819
0.671563
0.9615024
0.8036096
1.0
0.7696757
0.715254
0.7774255
0.95004386
0.8139935
1.0
0.92240244
0.9754132
0.98807496
0.95590377
0.9097107
1.0
0.4995226
0.59284717
0.8752348
0.8818971
1.0
0.80171067
0.7117474
0.8809638
0.96758544
1.0
0.75019544
0.52813494
0.88125056
0.98190033
1.0
0.6068049
0.9539993
0.8630381
0.9852293
1.0
0.99078256
0.6550196
0.8565827
0.9725543
1.0
0.48913103
0.74009633
0.81398445
0.89291024
1.0
0.50885564
0.7532552
0.84262615
0.84222263
0.8730099
1.0
0.41131833
0.78194004
0.9021438
0.809429
0.96898514
1.0
0.80769277
0.7073718
0.98290414
0.81074077
0.9479976
1.0
0.61390436
0.89452255
0.7133365
0.8821718
1.0
0.99306047
0.867015
0.8009753
0.8812447
0.90572184
1.0
0.99814117
0.71850437
0.85852873
0.78789955
0.86154246
1.0
0.48635802
0.755616

1.0
0.70426196
0.979253
0.8809072
1.0
0.5040753
0.9626173
0.8674632
1.0
0.9021645
0.9894555
0.93391776
1.0
0.7270792
0.9633659
0.9203533
1.0
0.7342532
0.9763615
0.9164876
1.0
0.73655045
0.9889986
0.87001055
1.0
0.99108434
0.7299583
0.96357393
0.9156684
1.0
0.7593865
0.98704094
0.9191305
1.0
0.72404265
0.985614
0.9176164
1.0
0.59597594
0.96449304
0.9255331
1.0
0.99438804
0.9985147
0.908488
0.95410264
0.9064206
1.0
0.99937683
0.99880683
0.9014718
0.94954765
0.90196073
1.0
0.5523114
0.9768092
0.91730404
1.0
0.99650854
0.99870026
0.62945443
0.98454154
0.8976462
1.0
0.9930056
0.650631
0.99267733
0.9454212
1.0
0.99497557
0.9942924
0.9986915
0.6265708
0.9571371
0.9027685
1.0
0.98871046
0.9986558
0.5171964
0.98794496
0.8954613
1.0
0.98785174
0.998346
0.66032547
0.98807913
0.9260511
1.0
0.98800886
0.99857116
0.58376807
0.986919
0.8660947
1.0
0.9940753
0.99847645
0.57722837
0.9853453
0.9238196
1.0
0.9925289
0.99789125
0.64766914
0.97294486
0.91757756
1.0
0.99776995
0.6121476
0.898478
1.0
0.99873

0.8975209
0.9999795
0.9263281
0.99996805
0.97982293
0.99998033
0.97519004
0.9999709
0.94337
0.9999733
0.97633547
0.99998486
0.9784474
0.8453863
0.9999707
0.9621669
0.78692555
0.9999832
0.9587082
0.8891868
0.99998474
0.9578916
0.9142415
0.9999702
0.96468616
0.93829983
0.99997294
0.96721786
0.9511045
0.9999796
0.94652337
0.91433084
0.99998415
0.96913517
0.9337345
0.99998355
0.9730601
0.8747973
0.9999784
0.98728454
0.8496956
0.99999225
0.9866106
0.84573936
0.9999794
0.9785816
0.92439955
0.99998856
0.97808194
0.93938136
0.9999908
0.9640323
0.8774532
0.9999796
0.96987355
0.93398386
0.9999881
0.9287548
0.5895657
0.953711
0.9999927
0.97632706
0.99997663
0.9564072
0.9999813
0.96896356
0.9999777
0.9657796
0.9999865
0.74034977
0.63329494
0.999987
0.9535698
0.9999926
0.9557429
0.9999826
0.95751995
0.9999893
0.9602271
0.9999912
0.9501688
0.9999907
0.9483
0.9999794
0.93554527
0.9999882
0.9669153
0.99999416
0.9549822
0.9999913
0.9554093
0.99998105
0.9455807
0.9999865
0.9548246
0.5143541
0.9999765
0.

0.97629833
0.96343285
0.98052204
0.8216163
0.9962534
0.997361
0.9918128
0.78583974
0.9947397
0.8792943
0.999616
0.71940565
0.9917991
0.7754035
0.811594
0.9972549
0.63835067
0.9790252
0.9869017
0.7599929
0.99846905
0.55502117
0.991193
0.749949
0.9990301
0.47734702
0.9873958
0.7164676
0.9949963
0.90730643
0.9281586
0.9953324
0.8625663
0.99581534
0.837905
0.8761765
0.99591947
0.37768584
0.8408241
0.9990607
0.49646258
0.5298627
0.8595301
0.97835094
0.9650751
0.5205042
0.8630145
0.9968526
0.6515601
0.5852715
0.6339935
0.996931
0.6967695
0.64840007
0.8276676
0.6570972
0.9469526
0.75399095
0.74620926
0.74206036
0.9965749
0.72810835
0.5592883
0.74870455
0.9991423
0.9750812
0.99828905
0.7426423
0.99579084
0.4478246
0.67827594
0.86096036
0.83952904
0.9640367
0.984195
0.77742696
0.7561527
0.9963737
0.9889914
0.8690056
0.9740287
0.74240017
0.99880016
0.97878075
0.8791421
0.9919505
0.78054297
0.99853086
0.8983801
0.5445417
0.985429
0.69962275
0.99734914
0.790728
0.6208264
0.991916
0.5748769
0.99989

0.8661492
0.9734321
0.99990773
0.99673945
0.57039624
0.6193636
0.9913947
0.9998827
0.8721389
0.56495994
0.88034713
0.9935021
0.9998952
0.9868528
0.67082554
0.9785387
0.98526496
0.99966264
0.4739296
0.66785544
0.86170965
0.9997669
0.51926774
0.7665985
0.7496794
0.9992939
0.418387
0.6458721
0.69883513
0.9935592
0.99970454
0.43601787
0.5793611
0.82096386
0.9980223
0.99983084
0.56432575
0.45539233
0.5272807
0.9983424
0.9999461
0.6863693
0.6052693
0.88436705
0.9981646
0.9997954
0.64358604
0.8545009
0.9968623
0.99956626
0.7345765
0.95105344
0.99796873
0.993119
0.9994917
0.49414143
0.8058404
0.96161586
0.99213773
0.9998586
0.60057884
0.5992568
0.9003356
0.97700924
0.9870443
0.99957746
0.5131051
0.677853
0.9967319
0.9391339
0.9976114
0.99981624
0.30458334
0.5968583
0.9295379
0.99583757
0.9998049
0.491662
0.61690265
0.6851678
0.9982926
0.999936
0.5324847
0.5384911
0.94318485
0.9967721
0.9998486
0.596474
0.59748596
0.94056314
0.9971814
0.9998574
0.45708945
0.5632192
0.9821162
0.9971167
0.9915109

0.80281395
0.74010223
0.70366925
0.95748574
0.7791251
0.8881245
0.6082232
0.9751752
0.70253164
0.835664
0.96991146
0.9646828
0.99647754
0.7095944
0.7877547
0.98144275
0.82677746
0.9899857
0.4820384
0.9709825
0.93578106
0.9482777
0.9660758
0.503309
0.96332854
0.94471043
0.9172926
0.48299906
0.9634221
0.95571244
0.99969566
0.62460846
0.96589667
0.66292554
0.9986767
0.5560591
0.9688116
0.5100422
0.9933883
0.48454612
0.96903944
0.48861474
0.990471
0.5214079
0.73345906
0.7847521
0.89179176
0.6452726
0.6891676
0.6839111
0.9988423
0.61231697
0.94009537
0.54477316
0.9988423
0.62914157
0.9434244
0.6851951
0.5186836
0.9989103
0.88089406
0.9335936
0.5349305
0.5967925
0.7033428
0.9294386
0.8996918
0.7448607
0.69322866
0.9823735
0.9659417
0.6667391
0.66937524
0.9912822
0.9878483
0.6236765
0.6309476
0.96572554
0.99461526
0.97222507
0.5900873
0.97679615
0.9540997
0.98332626
0.6076001
0.98208505
0.99683493
0.98072666
0.69452286
0.943661
0.983763
0.983471
0.64274424
0.947909
0.9926851
0.9800829
0.66412

AttributeError: 'NoneType' object has no attribute 'copy'